In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

## Import libraries

In [ ]:
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### Model - LTC

In [ ]:
class LTC(nn.Module):
    
    """
    The LTC network.
    """

    def __init__(self, input_size, opt, d, pretrained=False):
      
        """LTC Model"""
        
        super(LTC, self).__init__()
        self.opt = opt
        
        self.LTC_conv1 = nn.Conv3d(opt[0], 64, kernel_size=(3, 3, 3), 
                                   padding=(1, 1, 1))
        self.LTC_pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        
        self.LTC_conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), 
                                   padding=(1, 1, 1))
        self.LTC_pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        
        self.LTC_conv3 = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), 
                                   padding=(1, 1, 1))
        self.LTC_pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        
        self.LTC_conv4 = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), 
                                   padding=(1, 1, 1))
        self.LTC_pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        
        self.LTC_conv5 = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), 
                                   padding=(1, 1, 1))
        self.LTC_pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        
        oT = math.floor(opt[1]/16); # 4 times max pooling of 1/2
        oH = math.floor(opt[2]/32); # 5 times max pooling of 1/2
        oW = math.floor(opt[3]/32); # 5 times max pooling of 1/2

        self.LTC_fc1 = nn.Linear(256*oT*oH*oW, 2048)
        self.LTC_fc2 = nn.Linear(2048, 2048)
        
        
        """Regression Model"""
        
        self.Reg_fc1 = nn.Linear(input_size, 512)
        self.Reg_fc2 = nn.Linear(512, 1024)
        
        
        """Combined LTC + Regression feature maps"""
        
        self.Merged_fc1 = nn.Linear(3072, 1024)
        self.batchnorm1 = nn.BatchNorm1d(1024)
        
        self.Merged_fc2 = nn.Linear(1024, 256)
        self.batchnorm2 = nn.BatchNorm1d(256)

        self.Merged_fc3 = nn.Linear(256, 64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
        self.Merged_fc4 = nn.Linear(64, 1)
        
        self.relu = nn.ReLU()
      
        self.dropout_video = nn.Dropout(0.7)
        self.dropout_meta  = nn.Dropout(0.5)
        

    def forward(self, x, metadata):
      
        """ Training - LTC (Video) """
        
        x_conv1 = self.relu(self.LTC_conv1(x))
        x_pool1 = self.LTC_pool1(x_conv1)
        x_pool1 = self.dropout_video(x_pool1)

        x_conv2 = self.relu(self.LTC_conv2(x_pool1))
        x_pool2 = self.LTC_pool2(x_conv2)
        x_pool2 = self.dropout_video(x_pool2)
        
        x_conv3 = self.relu(self.LTC_conv3(x_pool2))
        x_pool3 = self.LTC_pool3(x_conv3)
        x_pool3 = self.dropout_video(x_pool3)
        
        x_conv4 = self.relu(self.LTC_conv4(x_pool3))
        x_pool4 = self.LTC_pool4(x_conv4)
        x_pool4 = self.dropout_video(x_pool4)
        
        x_conv5 = self.relu(self.LTC_conv5(x_pool4))
        x_pool5 = self.LTC_pool5(x_conv5)
        x_pool5 = self.dropout_video(x_pool5)
        
        oT = math.floor(self.opt[1]/16); # 4 times max pooling of 1/2
        oH = math.floor(self.opt[2]/32); # 5 times max pooling of 1/2
        oW = math.floor(self.opt[3]/32); # 5 times max pooling of 1/2

        x_fc = x_pool5.view(x_pool5.size()[0],256*oT*oH*oW)
        x_fc1 = self.relu(self.LTC_fc1(x_fc))
        x_fc2 = self.relu(self.LTC_fc2(x_fc1))

        
        """Training - Regression (Metadata)"""
        
        metadata_fc1 = self.relu(self.Reg_fc1(metadata))
        metadata_fc1_dropout = self.dropout_meta(metadata_fc1)
        metadata_fc2 = self.relu(self.Reg_fc2(metadata_fc1_dropout))
        
        
        """Concatenated LTC + Regression feature maps"""
        
        flattened_layer = torch.cat((x_fc2, metadata_fc2), 1)
        merged_fc1 = self.relu(self.Merged_fc1(flattened_layer))   
        merged_fc1_dropout = self.dropout_meta(merged_fc1)
       
        merged_fc2 = self.relu(self.Merged_fc2(merged_fc1_dropout))   
        merged_fc2_dropout = self.dropout_meta(merged_fc2)
        
        merged_fc3 = self.relu(self.Merged_fc3(merged_fc2_dropout))   
        merged_fc3_dropout = self.dropout_meta(merged_fc3)
        
        result = self.Merged_fc4(merged_fc3_dropout)
        
        return result

In [ ]:
def train(traintensor, label, metadata, batchsize):
    model.train()
    traintensor = traintensor.cpu().data.numpy()
    metadata = metadata.cpu().data.numpy()
    nbatch = traintensor.shape[0] / batchsize
    split_tensor = np.array_split(traintensor, nbatch)
    split_data = np.array_split(metadata, nbatch)
    start, end = 0, 0
    trainloss_batch = []
    prediction = []
    for pos in range(0, len(split_tensor)):
        optimizer.zero_grad()
        output = model.forward(torch.FloatTensor(split_tensor[pos]).cuda(), torch.FloatTensor(split_data[pos]).cuda())
        start = end
        end = start + split_tensor[pos].shape[0]
        L = loss(output, torch.tensor(label[start:end]))
        L.backward()
        optimizer.step()
        trainloss_batch.append(L.item())
        prediction.extend([output.tolist(), label[start:end]])
    print("Train Loss: ", L.item())
    return np.average(trainloss_batch), prediction


def test(testtensor, label, metadata, batchsize):
    model.eval()
    testtensor = testtensor.cpu().data.numpy()
    metadata = metadata.cpu().data.numpy()
    nbatch = testtensor.shape[0] / batchsize
    split_tensor = np.array_split(testtensor, nbatch)
    split_data = np.array_split(metadata, nbatch)
    start, end = 0, 0
    testloss_batch = []
    prediction = []
    for pos in range(0, len(split_tensor)):
        output = model.forward(torch.FloatTensor(split_tensor[pos]).cuda(), torch.FloatTensor(split_data[pos]).cuda())
        start = end
        end = start + split_tensor[pos].shape[0]
        L = loss(output, torch.tensor(label[start:end]))
        testloss_batch.append(L.item())
        prediction.extend([output.tolist(), label[start:end]])
    print("Test Loss: ", np.average(testloss_batch))
    return np.average(testloss_batch), prediction


def split_tensor(tensor, channels, timeDepth, xSize, ySize, train_id,
                 test_id):
    traintensor = torch.FloatTensor(len(train_id), channels,
                                    timeDepth, xSize, ySize)
    testtensor = torch.FloatTensor(len(test_id), channels,
                                   timeDepth, xSize, ySize)

    for i in range(len(traintensor)):
        traintensor[i] = tensor[train_id[i]]

    for i in range(len(testtensor)):
        testtensor[i] = tensor[test_id[i]]

    return traintensor, testtensor


def split_metadata(X, Y, train_id, test_id):
    return X[train_id], X[test_id], Y[train_id], Y[test_id]


def save_model(model, output_modelpath, e, trainLoss, testLoss):
    torch.save(model.state_dict(), newmodelpath + str(e + 1) + ".pt")
    np.savetxt(output_modelpath+'train'+str(e+1)+'.out', np.array(trainLoss), delimiter=',')
    np.savetxt(output_modelpath+'test'+str(e+1)+'.out', np.array(testLoss), delimiter=',')
    
    
def plot_lossgraph(loss, loss_=None):
    fig, axs = plt.subplots(1, 1, figsize=(10, 10))
    fig.suptitle(loss_)
    axs.plot(loss)
    axs.plot(loss, "o")
    axs.set_ylabel('Loss')
    axs.set_xlabel('Epochs')
    plt.show()

In [ ]:
if __name__ == "__main__":

    train_metadatapath = "gdrive/My Drive/Project - Box Office Prediction/Dataset/Metadata/Train_2382.csv"
    test_metadatapath = "gdrive/My Drive/Project - Box Office Prediction/Dataset/Metadata/Test_298.csv"
    output_modelpath = "gdrive/My Drive/Project - Box Office Prediction/Mehmood/Model/LTC_0318/"
    # modelpath = output_modelpath+"2100.pt"    #Load a pre-trained model to resume the training
    timeDepth = 16
    channels = 6
    xSize, ySize = 58, 58
    trainLoss = []
    testLoss = []
    epochs = 2000
    checkpoint_epoch = 1

    print("Checking runtime configuation...")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Runtime: ", device)
    if device == "cpu":
        print("Change the runtime to GPU")
        exit(0)
        
    """Process Metadata"""
    
    print("Processing metadata...")
    train_data = pd.read_csv(train_metadatapath)
    test_data = pd.read_csv(test_metadatapath)
    drop_columns = ['videoname']
    
    train_data = train_data.drop(columns=drop_columns)
    test_data = test_data.drop(columns=drop_columns)
    
    Y_train = train_data['revenue'].values
    train_data = train_data.drop(columns='revenue')
    X_train = train_data.values
    Y_test = test_data['revenue'].values
    test_data = test_data.drop(columns='revenue')
    
    X_test = test_data.values
    tr_datacolumns = train_data.columns
    tr_num_cols = []
    tst_datacolumns = test_data.columns
    tst_num_cols = []
    for i in range(0,len(tr_datacolumns)):
        if(np.max(train_data[tr_datacolumns[i]]) > 1):
            tr_num_cols.append(i)
    for i in range(0,len(tst_datacolumns)):
        if(np.max(test_data[tst_datacolumns[i]]) > 1):
            tst_num_cols.append(i)

    scaler_X = MinMaxScaler()
    X_train[:, tr_num_cols] = scaler_X.fit_transform(X_train[:, tr_num_cols])
    X_test[:, tr_num_cols] = scaler_X.transform(X_test[:, tr_num_cols])
    scaler_Y = MinMaxScaler()
    Y_train = scaler_Y.fit_transform(Y_train.reshape(-1, 1))
    Y_test = scaler_Y.transform(Y_test.reshape(-1, 1))
    
    print(X_train.shape, X_test.shape)

    print("Metadata processed successfully...")

    """Process Trailers"""
    
    print("Loading Trailer Tensors... ")
    trailertensor1 = torch.load('gdrive/My Drive/LTCTensor/TrailerTensor_1489_augmented_train.pt')
    trailertensor2 = torch.load('gdrive/My Drive/LTCTensor/TrailerTensor_1489_augmented_train1.pt')
    traintensor = torch.cat((trailertensor1, trailertensor2), 0)
    testtensor = torch.load('gdrive/My Drive/LTCTensor/TrailerTensor_1489_augmented_test.pt')
    print("Trailer Tensors Loaded Successfully...")

    model = LTC(input_size=X_train.shape[1],
              opt=[channels, timeDepth, xSize, ySize], d=0.5, pretrained=True)
    optimizer = optim.Adam(model.parameters(), lr=1e-7,
                          betas=(0.9, 0.999), eps=1e-08)
    loss = nn.MSELoss()

    if torch.cuda.is_available():
        model = torch.nn.DataParallel(model)
        print("Model: ", type(model))
        print("Devices: ", model.device_ids)
        model = model.cuda()
        # model.load_state_dict(torch.load(modelpath))
        print('here')
        loss = loss.cuda()
        traintensor = traintensor.cuda()
        testtensor  = testtensor.cuda()
        X_train     = torch.FloatTensor(X_train).cuda()
        Y_train     = torch.FloatTensor(Y_train).cuda()
        X_test      = torch.FloatTensor(X_test).cuda()
        Y_test      = torch.FloatTensor(Y_test).cuda()

    print("Training the model...")
    for e in range(epochs):
        print("\nEpoch: ", e + 1, "/", epochs)
        batchsize = 40
        optimizer.zero_grad()
        trLoss, pred_train = train(traintensor, Y_train,
                      X_train, batchsize)
        trainLoss.append(trLoss)
        tstLoss, prediction = test(testtensor, Y_test,
                X_test, batchsize)
        testLoss.append(tstLoss)
        pred = []
        pred_tr = []
        for pr in range(len(prediction)):
            if pr%2==0:
                pred.extend(prediction[pr])

        for pr in range(len(pred_train)):
            if pr%2==0:
                pred_tr.extend(pred_train[pr])    

        Y_t = scaler_Y.inverse_transform(Y_test.cpu().data.numpy())
        p_t = scaler_Y.inverse_transform(np.array(pred).reshape(-1,1))
        Y_tr = scaler_Y.inverse_transform(Y_train.cpu().data.numpy())
        p_tr = scaler_Y.inverse_transform(np.array(pred_tr).reshape(-1,1))

        print("r2_train", r2_score(Y_tr, p_tr))
        print("r2_test", r2_score(Y_t, p_t))

        if (e + 1) % checkpoint_epoch == 0:
            # save_model(model, output_modelpath, e, trainLoss, testLoss)
            plot_lossgraph(trainLoss, "Training Loss")
            plot_lossgraph(testLoss, "Testing Loss")
            
    print("Training compeleted...")

